# Get metadata for records containing spreadsheet files in Zenodo
first up, following the instructions on the [Zenodo developers site](https://developers.zenodo.org/?python#quickstart-upload) and this [instruction for looping through API page results](https://viniciusdesouza-becker.medium.com/making-a-request-in-a-paginated-api-using-python-7be6279c2aca)

In [1]:
import requests
import re
from datetime import datetime
import json

In [ ]:
#enter the Zenodo API access token
#ACCESS_TOKEN = input("enter Zenodo access token: ")

In [2]:
#set up some lists
total_results = []
record_id_list = []
#set up current date and time for output filename
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

In [3]:
#set up API details
url = 'https://zenodo.org/api/records'
params = {'q': 'filetype:(xlsb) AND doi:zenodo',
        'size': '20',
        #'access_token': ACCESS_TOKEN
        # 'q': 'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xlsb) AND doi:zenodo'
       }

In [9]:
# While results['links']['next'] isn't empty, let's download the next page, too
#ran into problems, looked at https://stackoverflow.com/questions/50708172/keyerror-in-while-loop
#still having problems, trying https://farnamdata.com/api-pagination/
#what finally worked was this: https://viniciusdesouza-becker.medium.com/making-a-request-in-a-paginated-api-using-python-7be6279c2aca

#a while loop that gets all the pages of API results using the 'next' value in the result
while True:
    response = requests.get(url, params = params)
    print(f'current page {response.url}')
    if response.status_code != 200:
        raise ValueError('Request failed with status code ' + str(response.status_code)) 
    results = response.json()
    print(f'the links on this current page are: {results["links"]}')
    total_results.extend(results['hits']['hits'])
    print(f'{len(results["hits"]["hits"])} records were added to the big list')
    if 'next' in results['links'].keys():
        url = results['links']['next']
        print(f'setting next page: {url}')
    else:
        break

current page https://zenodo.org/api/records?q=filetype%3A%28xlsb%29+AND+doi%3Azenodo&size=20
the links on this current page are: {'self': 'https://zenodo.org/api/records?page=1&q=filetype%3A%28xlsb%29%20AND%20doi%3Azenodo&size=20&sort=bestmatch', 'next': 'https://zenodo.org/api/records?page=2&q=filetype%3A%28xlsb%29%20AND%20doi%3Azenodo&size=20&sort=bestmatch'}
20 records were added to the big list
setting next page: https://zenodo.org/api/records?page=2&q=filetype%3A%28xlsb%29%20AND%20doi%3Azenodo&size=20&sort=bestmatch
current page https://zenodo.org/api/records?page=2&q=filetype%3A%28xlsb%29%20AND%20doi%3Azenodo&size=20&sort=bestmatch&q=filetype%3A%28xlsb%29+AND+doi%3Azenodo&size=20
the links on this current page are: {'prev': 'https://zenodo.org/api/records?page=1&q=filetype%3A%28xlsb%29%20AND%20doi%3Azenodo&size=20&sort=bestmatch', 'self': 'https://zenodo.org/api/records?page=2&q=filetype%3A%28xlsb%29%20AND%20doi%3Azenodo&size=20&sort=bestmatch', 'next': 'https://zenodo.org/api/re

In [10]:
print("We have", len(total_results), "total results")

We have 48 total results


In [ ]:
#output metadata records to one giant file
#explore: also output pickle file?
with open(f'zenodo_API_metadata_results_{filetime}.json', 'w') as f:
    json.dump(total_results, f)

In [ ]:
#loop through the metadata records to extract the record id values
#also write individual metadata files for each record
for record in total_results:
    record_id = record['id']
    #put the id in a text file or some object that will then become the text file
    record_id_list.append(record_id)
    #write out a json metadata file for a single record, put it in a directory named for the id?
    with open(f'{record_id}_metadata_{filetime}.json', 'w') as f:
       json.dump(record, f)

In [ ]:
#make a text file with a record id on each line, so zenodo_get can be used later to download files
#https://pytutorial.com/python-dump-list-to-file/    
print(record_id_list)    
with open(f'record_ids_{filetime}.txt', 'w') as file:
    for item in record_id_list:
        file.write(f"{item}\n")

The rest of this is various code that failed or I don't need anymore

In [ ]:
# print('downloading first page of search results')
# response = requests.get(url, params = params)
# print(response.url)
# print(response.links)
# results = response.json()

In [ ]:
# print(results['links'].keys())
# if 'next' in results['links'].keys():
#     print("yay")
# else:
#     print("boo")

In [ ]:
# # Store the first page of results
# total_results = total_results + results['hits']['hits']
# #print(total_results)

In [ ]:
# response = requests.get(results['links']['next'])
# results = response.json()
# print(response.url)
# # Store the current page of results
# total_results = total_results + results['hits']['hits']
# #print(total_results)

In [ ]:
#from https://jonathansoma.com/lede/foundations-2018/classes/apis/multiple-pages-of-data-from-apis/
# # Start with an empty list
# total_results = []

# # Grab the search results
# print("Downloading the original search results")
# response = requests.get("https://swapi.co/api/people/?search=a")
# data = response.json()

# # Store the first page of results
# total_results = total_results + data['results']

# # While data['next'] isn't empty, let's download the next page, too
# while data['next'] is not None:
#     print("Next page found, downloading", data['next'])
#     response = requests.get(data['next'])
#     data = response.json()
#     # Store the current page of results
#     total_results = total_results + data['results']

# print("We have", len(total_results), "total results")

In [ ]:
# response = requests.get('https://zenodo.org/api/records',
#                         params={'q': 'filetype:(csv OR xlsx OR tsv OR xls OR ods OR xslb) AND doi:zenodo',
#                                 'size': '10',
#                                 #'access_token': ACCESS_TOKEN
#                                 # regular search: https://zenodo.org/search?q=&f=file_type%3Axlsx&f=file_type%3Acsv&f=file_type%3Atsv&f=file_type%3Axls&f=file_type%3Axlsb&f=file_type%3Aods&l=list&p=1&s=10&sort=newest
#                                })
# results = response.json()

In [ ]:
# with open('spreadsheet-metadata_%s.json' %filetime, 'w') as f:
#     f.write(response.text)

In [ ]:
# print(response.url)

In [ ]:
# for record in results['hits']['hits']:
#     record_id = record['id']
#     with open('metadata_%s.json' %record_id, 'w') as f:
#         json.dump(record, f, ensure_ascii=False)
#     # # print(record['metadata']['title'])
#     # # print(record['files'])
#     # for file in record['files']:
#     #     filename = file['key']
#     #     #print(filename)
#     #     if re.search(r'\.(csv|tsv|xlsx|xls|xlsb|ods)', filename):
#     #         print("spreadsheet file found:", filename)

In [ ]:
# len("Output_Area_to_LSOA_to_MSOA_to_Local_Authority_District__December_2017__Lookup_with_Area_Classifications_in_Great_Britain.csv")

In [ ]:
# for filetype in results['aggregations']['file_type']['buckets']:
#     print(filetype['key'],filetype['doc_count'])